In [9]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [18]:
# ## RUN THIS ONLY ONCE WHEN YOU HAVE NEW DATA

# # Function to add the feature descriptions as column names to a DataFrame
# def add_feature_descriptions(speaker_df, descriptions_df):
#     # Set the new column headers
#     descriptions_df.columns = ['feature_name', 'feature_description']
#     # Use the 'feature_name' column as the column names for speaker_df
#     speaker_df.columns = descriptions_df['feature_name'].values
#     return speaker_df


# # Function to add the labels to the speaker DataFrame
# def add_labels(speaker_df, labels_df, participant_id):
#     # Find the matching row in the labels DataFrame by participant ID
#     labels_row = labels_df.loc[labels_df['Participant_ID'] == participant_id]
#     # If the row is found, add the Depression and Gender values as new columns
#     if not labels_row.empty:
#         speaker_df['Depression'] = labels_row['Depression'].values[0]
#         speaker_df['Gender'] = labels_row['Gender'].values[0]
#         speaker_df['Participant_ID'] = participant_id
#     return speaker_df

# # # Paths to CSV files
# features_description_path = '/Users/Ayan/Desktop/CS5622/HW5/feature_description.csv'
# labels_path = '/Users/Ayan/Desktop/CS5622/HW5/labels.csv'
# features_train_dir = '/Users/Ayan/Desktop/CS5622/HW5/features_train'
# features_test_dir = '/Users/Ayan/Desktop/CS5622/HW5/features_test'


# # # Read the feature descriptions and labels CSV files
# features_description_df = pd.read_csv(features_description_path, encoding='latin-1', header=None)
# labels_df = pd.read_csv(labels_path, encoding='latin-1')

# # Iterate over each speaker CSV file in the features_train directory
# for file in os.listdir(features_test_dir):
#     if file.endswith('.csv'):
#         # Extract the participant ID from the file name
#         participant_id = int(file.split('_')[1].split('.')[0])
#         # Read the speaker CSV file
#         speaker_df = pd.read_csv(os.path.join(features_test_dir, file), encoding='latin-1')
        
#         # Add feature descriptions as column headers
#         speaker_df = add_feature_descriptions(speaker_df, features_description_df)
        
#         # Add labels based on participant ID
#         speaker_df = add_labels(speaker_df, labels_df, participant_id)
        
#          # make sure the dataframe is updated and the file itself is updated too
#         speaker_df.to_csv(os.path.join(features_test_dir, file), index=False)


In [20]:
# function to aggregate all training csv's into one big training data table
def aggregate_training_data(features_train_dir):
    # Create an empty DataFrame to store the aggregated data
    aggregated_df = pd.DataFrame()
    
    # Iterate over each speaker CSV file in the features_train directory
    for file in os.listdir(features_train_dir):
        if file.endswith('.csv'):
            # Read the speaker CSV file
            speaker_df = pd.read_csv(os.path.join(features_train_dir, file), encoding='latin-1')
            # Append the speaker data to the aggregated DataFrame
            aggregated_df = pd.concat([aggregated_df, speaker_df], ignore_index=True)
    
    return aggregated_df


# Aggregate all training data into one DataFrame
training_data = aggregate_training_data(features_train_dir)
testing_data = aggregate_training_data(features_test_dir)

# Save the aggregated training data to a CSV file
training_data.to_csv('/Users/Ayan/Desktop/CS5622/HW5/training_data.csv', index=False)
testing_data.to_csv('/Users/Ayan/Desktop/CS5622/HW5/testing_data.csv', index=False)

### RNN For Depression Detection

In [27]:
# read in the training data
training_data = pd.read_csv('/Users/Ayan/Desktop/CS5622/HW5/training_data.csv', encoding='latin-1')

# print the first 5 rows of the training data
training_data.head()

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,Depression,Gender,Participant_ID
0,23.442284,0.015231,23.083265,23.519197,23.799660,0.716394,0.000000,0.000000,-0.109734,0.000000,...,3.703704,2.040816,0.080000,0.000000,0.380,0.000000,-47.326970,1,1,448
1,26.658195,0.095594,24.275490,27.404346,29.017082,4.741592,61.829530,67.673560,20.461290,11.705440,...,3.131991,2.036199,0.344444,0.235472,0.115,0.087321,-41.121784,1,1,448
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.333334,0.000000,0.000000,0.000000,0.240,0.000000,-56.265000,1,1,448
3,34.614662,0.005980,34.433628,34.559757,34.764160,0.330532,8.279264,5.828207,8.208370,6.596444,...,5.084746,1.886793,0.420000,0.000000,0.040,0.020000,-33.531155,1,1,448
4,43.782654,0.006530,43.540440,43.602890,44.168890,0.628452,7.757357,0.000000,3.900528,0.000000,...,2.531646,1.351351,0.110000,0.000000,0.300,0.010000,-39.635094,1,1,448


In [28]:
# data preprocessing

def interpolate(train):
    for column in train.columns:
        # If the column is not 'Depression', 'Gender', 'participant_id'
        if column not in ['Depression', 'Gender', 'Participant_ID']:
            train[column] = train[column].replace(0, np.nan)
            train[column] = train[column].interpolate(method='linear', limit_direction='forward', axis=0)
    return train

# find rows with outliers and remove them
# def remove_outliers(train):
#     # Calculate the z-scores of each feature in the training data
#     z_scores = (train.drop(columns=['Depression', 'Gender', 'Participant_ID']).apply(lambda x: np.abs((x - x.mean()) / x.std())))
#     # Find the rows with z-scores greater than 3
#     outliers = z_scores[(z_scores > 4).any(axis=1)]
#     # Remove the rows with outliers
#     train = train.drop(outliers.index)
#     return train

In [29]:

# interpolate the training data
training_data = interpolate(training_data)


In [37]:
X_train = training_data.drop(columns=['Depression', 'Gender', 'Participant_ID']) 
X_train = X_train.to_numpy()
# remove first row of the training data
y_train = training_data['Depression']

X_train.head()
X_test = testing_data.drop(columns=['Depression', 'Gender', 'Participant_ID'])
X_test = X_test.to_numpy()
# # remove first row of the testing data
# X_test = X_test.iloc[1:]
# y_test = testing_data['Depression']


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [38]:
# Assuming `X_train` and `X_test` are numpy arrays after proper preprocessing
# max_length should be the number of time steps in each sequence
# X_train.shape[1] should give you the number of features

print(X_train.shape)  # To check the shape of your input data
max_length = X_train.shape[1]  # This is the number of features in your dataset

# Define RNN architecture
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(None, max_length)))  # The 'None' here allows variable sequence length
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Since LSTM expects a 3D input of shape [samples, time steps, features]
# And each row in your data corresponds to one time step with multiple features
# You should reshape your data to add the time step dimension if not already done
# For example:
X_train = np.expand_dims(X_train, axis=1)  # This adds the time step dimension
X_test = np.expand_dims(X_test, axis=1)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64)

# Evaluate the model
predictions = (model.predict(X_test) > 0.5).astype(int)
test_accuracy = accuracy_score(y_test, predictions)


(13539, 88)


/Users/Ayan/Desktop/CS5622/HW5/myenv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6983 - loss: nan
Epoch 2/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7123 - loss: nan
Epoch 3/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7094 - loss: nan
Epoch 4/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7080 - loss: nan
Epoch 5/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7134 - loss: nan
Epoch 6/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7071 - loss: nan
Epoch 7/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7123 - loss: nan
Epoch 8/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7024 - loss: nan
Epoch 9/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7127 - loss: nan
Epoch 10/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7124 - loss: nan
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


ValueError: Found input variables with inconsistent numbers of samples: [3260, 3259]